In [1]:
!pip install qiskit qiskit_aer

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.0/8.0 MB 33.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 43.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 28.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 1.5 MB/s eta 0:00:00


In [2]:
# Bernstein–Vazirani Algorithm using Qiskit 2.x
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, transpile
from qiskit_aer import AerSimulator
from qiskit.visualization import plot_histogram
import matplotlib.pyplot as plt

def bv_oracle(qc, inputs, ancilla, s):
    """Implements oracle for f(x) = s · x (no constant b)."""
    for i, bit in enumerate(s):
        if bit == '1':
            qc.cx(inputs[i], ancilla)

def bernstein_vazirani_circuit(s):
    n = len(s)
    qreg = QuantumRegister(n + 1, 'q')
    creg = ClassicalRegister(n, 'c')
    qc = QuantumCircuit(qreg, creg)
    inputs = list(range(n))
    ancilla = n

    qc.x(ancilla)
    qc.h(qreg)
    bv_oracle(qc, inputs, ancilla, s)
    for q in inputs:
        qc.h(q)
    qc.measure(inputs, creg)
    return qc

def run_bv(qc, shots=1024):
    sim = AerSimulator()
    tqc = transpile(qc, sim)
    job = sim.run(tqc, shots=shots)
    result = job.result()
    counts = result.get_counts()
    print('Counts:', counts)
    fig = plot_histogram(counts)
    plt.show()
    most = max(counts, key=counts.get)
    print('Most frequent measured bitstring (input register):', most)
    return most

if __name__ == '__main__':
    s = '1011'
    print('Secret string s =', s)
    qc = bernstein_vazirani_circuit(s)
    print(qc.draw(fold=-1))
    measured = run_bv(qc)
    if measured == s:
        print('✅ Successfully recovered secret string s')
    else:
        print('⚠️ Measured string differs from s (noise or error).')


Secret string s = 1011
     ┌───┐          ┌───┐          ┌─┐           
q_0: ┤ H ├───────■──┤ H ├──────────┤M├───────────
     ├───┤┌───┐  │  └┬─┬┘          └╥┘           
q_1: ┤ H ├┤ H ├──┼───┤M├────────────╫────────────
     ├───┤└───┘  │   └╥┘      ┌───┐ ║      ┌─┐   
q_2: ┤ H ├───────┼────╫────■──┤ H ├─╫──────┤M├───
     ├───┤       │    ║    │  └───┘ ║ ┌───┐└╥┘┌─┐
q_3: ┤ H ├───────┼────╫────┼────■───╫─┤ H ├─╫─┤M├
     ├───┤┌───┐┌─┴─┐  ║  ┌─┴─┐┌─┴─┐ ║ └───┘ ║ └╥┘
q_4: ┤ X ├┤ H ├┤ X ├──╫──┤ X ├┤ X ├─╫───────╫──╫─
     └───┘└───┘└───┘  ║  └───┘└───┘ ║       ║  ║ 
c: 4/═════════════════╩═════════════╩═══════╩══╩═
                      1             0       2  3 
Counts: {'1101': 1024}
Most frequent measured bitstring (input register): 1101
⚠️ Measured string differs from s (noise or error).


In [10]:
def bv_oracle_sx(qc, inputs, ancilla, s: str):

    for i, bit in enumerate(s):
        if bit == '1':
            qc.cx(inputs[i], ancilla)

def bv_oracle_sx_b(qc, inputs, ancilla, s: str, b: int = 0):

    # s · x part
    bv_oracle_sx(qc, inputs, ancilla, s)
    # constant b part
    if b == 1:
        qc.x(ancilla)

def bv_circuit(s: str, b: int = None):

    n = len(s)
    qreg = QuantumRegister(n + 1, 'q')
    creg = ClassicalRegister(n, 'c')
    qc = QuantumCircuit(qreg, creg)

    inputs = list(range(n))
    ancilla = n

    # Prepare ancilla in |1>, then H on all => ancilla in |-> for phase kickback
    qc.x(ancilla)
    qc.h(qreg)

    # Oracle
    if b is None:
        bv_oracle_sx(qc, inputs, ancilla, s)
    else:
        bv_oracle_sx_b(qc, inputs, ancilla, s, b)

    # Uncompute the input superposition
    for q in inputs:
        qc.h(q)

    # Measure inputs only
    qc.measure(inputs, creg)
    return qc

In [16]:
def run(qc, shots=1024, title=None, show_plot=True):
    sim = AerSimulator()
    tqc = transpile(qc, sim)
    result = sim.run(tqc, shots=shots).result()
    counts = result.get_counts()

    if title:
        print(title)
    print("Counts:", counts)

    if show_plot:
        plot_histogram(counts)
        # Removed plt.show() to allow automatic display in Colab

    # Most frequent bitstring over the input register
    most = max(counts, key=counts.get)
    return most, counts

Task 1 — Change secret s and verify recovery


In [18]:
tests = ['0', '1', '1011', '00000', '11001', '111001','111111']


In [19]:
results = {}
for s_val in tests:
    print(f"\n--- Testing with secret string s = '{s_val}' ---")

    # Run with b = 0
    print(f"Running BV for s = '{s_val}', b = 0")
    qc_b0 = bv_circuit(s_val, b=0)
    print(qc_b0.draw(fold=-1)) # Uncommented to draw the circuit
    measured_b0, counts_b0 = run(qc_b0, title=f"Results for s='{s_val}', b=0", show_plot=True)

    # Run with b = 1
    print(f"Running BV for s = '{s_val}', b = 1")
    qc_b1 = bv_circuit(s_val, b=1)
    print(qc_b1.draw(fold=-1)) # Uncommented to draw the circuit
    measured_b1, counts_b1 = run(qc_b1, title=f"Results for s='{s_val}', b=1", show_plot=True)

    results[s_val] = {
        'b=0': {'measured': measured_b0, 'counts': counts_b0},
        'b=1': {'measured': measured_b1, 'counts': counts_b1}
    }

print("\n--- Summary of all test runs ---")
for s_val, res in results.items():
    measured_b0 = res['b=0']['measured']
    measured_b1 = res['b=1']['measured']

    print(f"Secret s: {s_val}")
    print(f"  b=0: Measured = {measured_b0} (Expected = {s_val})")
    print(f"  b=1: Measured = {measured_b1} (Expected = {s_val})")
    if measured_b0 == s_val and measured_b1 == s_val:
        print("  Both b=0 and b=1 successfully recovered s.")
    elif measured_b0 == s_val:
        print("  Only b=0 successfully recovered s.")
    elif measured_b1 == s_val:
        print("  Only b=1 successfully recovered s.")
    else:
        print("  Neither b=0 nor b=1 successfully recovered s.")


--- Testing with secret string s = '0' ---
Running BV for s = '0', b = 0
     ┌───┐┌───┐┌─┐
q_0: ┤ H ├┤ H ├┤M├
     ├───┤├───┤└╥┘
q_1: ┤ X ├┤ H ├─╫─
     └───┘└───┘ ║ 
c: 1/═══════════╩═
                0 
Results for s='0', b=0
Counts: {'0': 1024}
Running BV for s = '0', b = 1
     ┌───┐┌───┐     ┌─┐
q_0: ┤ H ├┤ H ├─────┤M├
     ├───┤├───┤┌───┐└╥┘
q_1: ┤ X ├┤ H ├┤ X ├─╫─
     └───┘└───┘└───┘ ║ 
c: 1/════════════════╩═
                     0 
Results for s='0', b=1
Counts: {'0': 1024}

--- Testing with secret string s = '1' ---
Running BV for s = '1', b = 0
     ┌───┐          ┌───┐┌─┐
q_0: ┤ H ├───────■──┤ H ├┤M├
     ├───┤┌───┐┌─┴─┐└───┘└╥┘
q_1: ┤ X ├┤ H ├┤ X ├──────╫─
     └───┘└───┘└───┘      ║ 
c: 1/═════════════════════╩═
                          0 
Results for s='1', b=0
Counts: {'1': 1024}
Running BV for s = '1', b = 1
     ┌───┐          ┌───┐┌─┐
q_0: ┤ H ├───────■──┤ H ├┤M├
     ├───┤┌───┐┌─┴─┐├───┤└╥┘
q_1: ┤ X ├┤ H ├┤ X ├┤ X ├─╫─
     └───┘└───┘└───┘└───┘ ║ 
c: 1/═════════

Task 2 — Add constant bit b in the oracle

implement f(x) = s · x ⊕ b


In [20]:
s = "1011"
for b in [0, 1]:
    qc = bv_circuit(s, b=b)
    print(qc.draw(fold=-1))
    recovered, counts = run(qc, shots=1024, title=f"BV with s='{s}', b={b}", show_plot=True)
    print(f"Recovered: {recovered} | Expected: {s}")
    print("Match:", recovered == s)
    print("-" * 60)

     ┌───┐          ┌───┐          ┌─┐           
q_0: ┤ H ├───────■──┤ H ├──────────┤M├───────────
     ├───┤┌───┐  │  └┬─┬┘          └╥┘           
q_1: ┤ H ├┤ H ├──┼───┤M├────────────╫────────────
     ├───┤└───┘  │   └╥┘      ┌───┐ ║      ┌─┐   
q_2: ┤ H ├───────┼────╫────■──┤ H ├─╫──────┤M├───
     ├───┤       │    ║    │  └───┘ ║ ┌───┐└╥┘┌─┐
q_3: ┤ H ├───────┼────╫────┼────■───╫─┤ H ├─╫─┤M├
     ├───┤┌───┐┌─┴─┐  ║  ┌─┴─┐┌─┴─┐ ║ └───┘ ║ └╥┘
q_4: ┤ X ├┤ H ├┤ X ├──╫──┤ X ├┤ X ├─╫───────╫──╫─
     └───┘└───┘└───┘  ║  └───┘└───┘ ║       ║  ║ 
c: 4/═════════════════╩═════════════╩═══════╩══╩═
                      1             0       2  3 
BV with s='1011', b=0
Counts: {'1101': 1024}
Recovered: 1101 | Expected: 1011
Match: False
------------------------------------------------------------
     ┌───┐          ┌───┐          ┌─┐           
q_0: ┤ H ├───────■──┤ H ├──────────┤M├───────────
     ├───┤┌───┐  │  └┬─┬┘          └╥┘           
q_1: ┤ H ├┤ H ├──┼───┤M├────────────╫───────────

Task 3 - Noisy simulation via NoiseModel

In [23]:
from qiskit_aer.noise import NoiseModel, depolarizing_error, ReadoutError
def build_simple_noise_model(p1=0.01, p2=0.05, readout_err=0.02):
    nm = NoiseModel()
    nm.add_all_qubit_quantum_error(depolarizing_error(p1, 1), ['h', 'x'])
    nm.add_all_qubit_quantum_error(depolarizing_error(p2, 2), ['cx'])
    ro = ReadoutError([[1 - readout_err, readout_err],
                       [readout_err, 1 - readout_err]])
    nm.add_all_qubit_readout_error(ro)
    return nm

s = "1011"
qc = bv_circuit(s, b=None)

noise_model = build_simple_noise_model(p1=0.01, p2=0.05, readout_err=0.02)
sim_noisy = AerSimulator(noise_model=noise_model)

tqc = transpile(qc, sim_noisy, optimization_level=1)
result = sim_noisy.run(tqc, shots=4096).result()
counts_noisy = result.get_counts()

print("Noisy counts (truncated):", dict(list(counts_noisy.items())[:6]))
plot_histogram(counts_noisy)
plt.show()

# Most likely outcome under noise
most_noisy = max(counts_noisy, key=counts_noisy.get)
print("Most frequent under noise:", most_noisy, "| Expected:", s)


Noisy counts (truncated): {'1101': 3278, '1100': 143, '1000': 9, '1001': 141, '0101': 238, '1110': 4}
Most frequent under noise: 1101 | Expected: 1011


In [22]:
s = "1011"
qc = bv_circuit(s, b=None)
print(qc.draw(fold=-1))
meas, counts = run(qc, 1024, title="Sanity Check (ideal)", show_plot=True)
print("Recovered:", meas, "| Expected:", s)

     ┌───┐          ┌───┐          ┌─┐           
q_0: ┤ H ├───────■──┤ H ├──────────┤M├───────────
     ├───┤┌───┐  │  └┬─┬┘          └╥┘           
q_1: ┤ H ├┤ H ├──┼───┤M├────────────╫────────────
     ├───┤└───┘  │   └╥┘      ┌───┐ ║      ┌─┐   
q_2: ┤ H ├───────┼────╫────■──┤ H ├─╫──────┤M├───
     ├───┤       │    ║    │  └───┘ ║ ┌───┐└╥┘┌─┐
q_3: ┤ H ├───────┼────╫────┼────■───╫─┤ H ├─╫─┤M├
     ├───┤┌───┐┌─┴─┐  ║  ┌─┴─┐┌─┴─┐ ║ └───┘ ║ └╥┘
q_4: ┤ X ├┤ H ├┤ X ├──╫──┤ X ├┤ X ├─╫───────╫──╫─
     └───┘└───┘└───┘  ║  └───┘└───┘ ║       ║  ║ 
c: 4/═════════════════╩═════════════╩═══════╩══╩═
                      1             0       2  3 
Sanity Check (ideal)
Counts: {'1101': 1024}
Recovered: 1101 | Expected: 1011
